# Colombia Results v2.2

Cargamos los resultados de iterar todos los conjuntos de indicadores

In [25]:
import pandas as pd
import numpy as np
import time
import plotly.plotly as py
import plotly.graph_objs as go

original = pd.read_csv("colombia_absolute_total_v2.csv").drop(['Unnamed: 0'], 1)

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
datos = datos[datos['CountryCode'] == 'COL']

paises = ['COL']
pais_nombre = 'Colombia'

preprocessing = 'imput'
search = 'original'
year_init = 2015
year_range = 20
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')


conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']
conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]

In [26]:
def estimadores_uso(df):

    df_1 = df
    temp = df['Results']
    estimadores = []

    for i in range(len(temp)):
        temp2 = temp[i] 
        r = temp2.replace(" '",",").replace("'","").replace("[[","").replace("]]","").split(']\n [')
        z = []

        for j in r:
            z.append(j.split(','))

        c = pd.DataFrame(z).groupby(1).count()[0]/len(z)

        estimadores.append([c.index.values.tolist(),c.values.tolist()])

    estimadores_df = pd.DataFrame(estimadores)
    estimadores_uso = pd.DataFrame(np.zeros((1200,4)),columns=['DTR','NONE','RFR','SVR'])

    for k in range(1200):
        for i in range(len(estimadores_df[0][k])):
            if (estimadores_df[0][k][i] == 'DTR'):
                estimadores_uso['DTR'][k] = estimadores_df[1][k][i]
            if (estimadores_df[0][k][i] == 'NONE'):
                estimadores_uso['NONE'][k] = estimadores_df[1][k][i]
            if (estimadores_df[0][k][i] == 'RFR'):
                estimadores_uso['RFR'][k] = estimadores_df[1][k][i]
            if (estimadores_df[0][k][i] == 'SVR'):
                estimadores_uso['SVR'][k] = estimadores_df[1][k][i]

    df_2 = estimadores_uso

    return pd.concat([df_1,df_2],axis=1)

In [27]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [28]:
def buscar_mejores(df):
    return df[(df.Resultados > 0.7)&(df.Resultados <1.0)].reset_index(drop=True)

In [29]:
def graficar_resultados(df,look_back,column,colorscale):

    trace = go.Heatmap(z=df[df['Look Back'] == look_back][column],
                       x=df[df['Look Back'] == look_back]['Base'],
                       y=df[df['Look Back'] == look_back]['Target'],
                       zmax=1.0,
                       zmin=0.0,
                       colorscale = colorscale)
    data=[trace]
    
    if(column == '%'):
        title='R2 Mean on '+pais_nombre+' with '+search+' parameters and Look Back of '+str(look_back)
    else:
        title='% of Use of ' +column+' on '+pais_nombre+' with '+search+' parameters and Look Back of '+str(look_back)
    
    layout = go.Layout(
        
        title=title,
        xaxis=dict(title='Base',   titlefont=dict(size=18)),
        yaxis=dict(title='Target', titlefont=dict(size=18))
        
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename=title)

In [30]:
def buscar_resultados(df,indicador_1,indicador_2,look_back):
    #Buscamos Resultados en base al conjunto de indicadores Base, Target y Look Back
    temp = df[(df['Base']==indicador_1)&(df['Target']==indicador_2)&(df['Look Back'] == look_back)]['Results'].values[0]

    r = temp.replace(" '",",").replace("'","").replace("[[","").replace("]]","").split(']\n [')
    z = []

    for j in r:
        z.append(j.split(','))

    num = pd.DataFrame(z)[0].values.tolist()
    
    results = []
    for i in num: 
        results.append(float(i))
    
    # Códigos de los indicadores con registros 
    df_nombres = pd.DataFrame(conjunto_nombre, columns=['Resultados'])
    indicador_1 = conjunto[df_nombres[df_nombres['Resultados']==indicador_1].index.values[0]]
    indicador_2 = conjunto[df_nombres[df_nombres['Resultados']==indicador_2].index.values[0]]     
    df = pd.DataFrame(indicador_2)
    
    # Filtro indicadores repetidos
    if(indicador_1!=indicador_2):
        indicador_2 = np.array(df.loc[~df.ix[:,0].isin(indicador_1)]).flatten()
    df_zeros = tabla_base(indicador_2).dropna(thresh=5,axis=1)
    icodes = df_zeros.columns.values

    # Buscamos Nombres de los Códigos de los Indicadores
    nombres = pd.DataFrame(datos).loc[:,['IndicatorName','IndicatorCode']].set_index('IndicatorCode').loc[icodes].drop_duplicates()

    # Concatenamos Resultados y Nombres, Reindexando en base a los nombres
    df_results = pd.DataFrame(np.column_stack([nombres,results]),columns=['Indicadores','Resultados'])
    
    return df_results

In [31]:
def graficar_resultados_individuales(df,indicador_base,indicador_target,look_back):

    df_temp = buscar_resultados(df,indicador_base,indicador_target,look_back)

    data = [go.Bar(
                x= df_temp['Indicadores'],
                y= df_temp['Resultados'],

        marker=dict(     
            color = df_temp['Resultados'].values,
            colorscale ='Viridis',
            showscale = True
            ),
        )]

    layout = go.Layout(
        title=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back),
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig, filename=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back))

In [32]:
def conjuntos_predecibles(df,look_back):
    df_temp = pd.DataFrame.pivot_table( df[df['Look Back'] == look_back].drop(['Time','Reg','Results','DTR','NONE','RFR','SVR'],1) , index=["Target"], columns=["Look Back"] ,aggfunc=[np.mean])
    return df_temp.sort_values([('mean','%',look_back)], ascending=False).reset_index()

In [33]:
original = estimadores_uso(original)
original

,Base,Target,Look Back,Reg,%,Time,Results,DTR,NONE,RFR,SVR
0,Agricultura,Agricultura,1,48,0.597859,24.733329,[[0.5585337486333002 'RFR' '0.503590106964']\n...,0.194444,0.027778,0.555556,0.222222
1,Agricultura,Agricultura,2,48,0.603123,25.641758,[[0.7451728188197096 'RFR' '0.57684803009']\n ...,0.111111,0.027778,0.666667,0.194444
2,Agricultura,Agricultura,3,48,0.587326,27.100648,[[0.5530661259572864 'RFR' '0.555593013763']\n...,0.166667,0.027778,0.527778,0.277778
3,Agricultura,Ambiente,1,96,0.390740,35.350856,[[0.20790200933165076 'RFR' '0.699298143387']\...,0.277778,0.018519,0.277778,0.425926
4,Agricultura,Ambiente,2,96,0.377481,36.842801,[[0.4879135950228819 'DTR' '0.02738904953']\n ...,0.185185,0.018519,0.425926,0.370370
5,Agricultura,Ambiente,3,96,0.372137,39.155785,[[0.2395945794721527 'RFR' '0.549711942673']\n...,0.092593,0.018519,0.500000,0.388889
6,Agricultura,Ayuda,1,71,0.344853,37.704790,[[1.0 'SVR' '0.0634689331055']\n [0.0 'RFR' '0...,0.183333,0.016667,0.366667,0.433333
7,Agricultura,Ayuda,2,71,0.355002,40.487791,[[1.0 'SVR' '0.070858001709']\n [0.80081445097...,0.133333,0.016667,0.366667,0.483333
8,Agricultura,Ayuda,3,71,0.379432,42.325126,[[1.0 'SVR' '0.0465610027313']\n [0.6878868926...,0.033333,0.016667,0.416667,0.533333
9,Agricultura,Ciencia,1,13,0.431338,10.398011,[[0.0 'RFR' '1.05632686615']\n [0.0 'RFR' '0.4...,0.416667,0.000000,0.583333,0.000000


In [34]:
def conjuntos_predictores(df,look_back):
    df_temp = pd.DataFrame.pivot_table( df[df['Look Back'] == look_back].drop(['Time','Reg','Results','DTR','NONE','RFR','SVR'],1) , index=["Base"], columns=["Look Back"] ,aggfunc=[np.mean])
    return df_temp.sort_values([('mean','%',look_back)], ascending=False).reset_index()

In [35]:
graficar_resultados(original,1,'DTR','Reds')

In [36]:
graficar_resultados(original,1,'RFR','Reds')

In [37]:
graficar_resultados(original,1,'SVR','Reds')

In [38]:
graficar_resultados(original,1,'NONE','Reds')

## Buscamos Mejores Resultados Individuales

In [39]:
buscar_mejores(buscar_resultados(original,'Agricultura','Infraestructura',1))

,Indicadores,Resultados
0,Container port traffic (TEU: 20 foot equivalen...,0.789257
1,Fixed telephone subscriptions,0.815154
2,"Improved water source, urban (% of urban popul...",0.849442
3,Internet users (per 100 people),0.81674
4,Mobile cellular subscriptions,0.739324
5,Mobile cellular subscriptions (per 100 people),0.735945


In [40]:
buscar_mejores(buscar_resultados(original,'Agricultura','Infraestructura',1))

,Indicadores,Resultados
0,Container port traffic (TEU: 20 foot equivalen...,0.789257
1,Fixed telephone subscriptions,0.815154
2,"Improved water source, urban (% of urban popul...",0.849442
3,Internet users (per 100 people),0.81674
4,Mobile cellular subscriptions,0.739324
5,Mobile cellular subscriptions (per 100 people),0.735945


## Mejores combinaciones

In [41]:
original.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results','DTR','NONE','RFR','SVR'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Urbano,Salud,2,149,0.673709
1,Urbano,Urbano,2,24,0.670529
2,Urbano,Salud,3,149,0.656035
3,Urbano,Salud,1,149,0.651880
4,Salud,Salud,1,152,0.649607
5,Salud,Salud,2,152,0.647476
6,Agricultura,Urbano,3,24,0.638974
7,Urbano,Urbano,3,24,0.635215
8,Urbano,Agricultura,3,48,0.623665
9,Urbano,Agricultura,2,48,0.623538


## Peores combinaciones

In [42]:
original.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results','DTR','NONE','RFR','SVR'],1).tail(10)

,Base,Target,Look Back,Reg,%
1190,Pobreza,Publico,1,97,0.045474
1191,Ambiente,Pobreza,2,24,0.044698
1192,Publico,Pobreza,2,25,0.042916
1193,Energia,Pobreza,2,25,0.038207
1194,Ambiente,Pobreza,3,24,0.036326
1195,Trabajo,Pobreza,1,25,0.032743
1196,Energia,Pobreza,1,25,0.032505
1197,Pobreza,Ciencia,3,13,0.029709
1198,Pobreza,Publico,3,97,0.025528
1199,Pobreza,Publico,2,97,0.024698


## Graficamos Resultados Individuales

Resultado de predecir el conjunto de indicadores de Infraestructura desde Agricultura

In [43]:
graficar_resultados_individuales(original,'Agricultura','Infraestructura',1)

In [44]:
graficar_resultados_individuales(original,'Infraestructura','Ciencia',1)

## Graficamos Resultados Globales

Este Heatmap nos muestra el rendimiento promedio del coeficiente de determinación para cada conjunto

In [45]:
graficar_resultados(original,1,'%','Viridis')

In [46]:
graficar_resultados(original,2,'%','Viridis')

In [47]:
graficar_resultados(original,3,'%','Viridis')

# Mejores Conjuntos Base y Target

In [48]:
pd.DataFrame(np.column_stack([conjuntos_predictores(original,1)[:5],conjuntos_predictores(original,1).tail(5),conjuntos_predecibles(original,1)[:5],conjuntos_predecibles(original,1).tail(5)]), columns=[['Predictor','Predictor','Predictor','Predictor','Predictable','Predictable','Predictable','Predictable'],['Best','R2','Worse','R2','Best','R2','Worse','R2']])

Predictor                                 Predictable            \
          Best        R2      Worse        R2         Best        R2   
0       Urbano  0.423038   Comercio  0.312533        Salud  0.518211   
1        Clima  0.379852  Educacion  0.303412  Agricultura  0.501277   
2        Salud  0.378003    Ciencia  0.292547        Clima  0.487286   
3     Economia  0.364835    Publico  0.268289       Urbano  0.470411   
4  Agricultura  0.362927    Pobreza  0.136245       Social   0.43941   

                         
       Worse         R2  
0   Comercio   0.253058  
1   Finanzas   0.251863  
2  Educacion   0.223129  
3    Pobreza   0.107487  
4    Publico  0.0776554